This notebook is used to save the tidal parameters calculated from the model output to a csv file in order to avoid having to load all the files run.

In [1]:
import os
import glob
import datetime
import matplotlib.pylab as plt
import matplotlib.ticker as ticker
from matplotlib.patches import Ellipse
import numpy as np
from IPython.display import display, Math, Latex
import csv
import pandas as pd
import re

import netCDF4 as nc
from scipy.optimize import curve_fit
from salishsea_tools import (viz_tools,tidetools, nc_tools)
from salishsea_tools.nowcast import (research_VENUS, analyze)

%matplotlib inline


#Hourly Data
##@ VENUS nodes
The functions below will facilitate loading the hourly data and writting the tidal parameters to a csv file.

In [2]:
def loadparam(to, tf, path, freq='h', depav='None'):
    """ This function loads all the data between the start and the end date
    that contains hourly velocity netCDF4 files. Then it mask, unstaggers and 
    rotates the velocities by component about the VENUS nodes. Lastly it fits 
    the velcities and caculates the tidal ellipse parameters for that date range.
    
    depth : vector [dmin, dmax]
    """
    if freq=='h':
        filesu = analyze.get_filenames(to,tf, '1h', 'grid_U', path)
        filesv=analyze.get_filenames(to,tf,'1h', 'grid_V', path)

        sites=research_VENUS.SITES
        i_c=sites['VENUS']['Central']['i']
        i_e=sites['VENUS']['East']['i']
        j_c=sites['VENUS']['Central']['j']
        j_e=sites['VENUS']['East']['j']
        
        a = filesu
        b = filesv
        c = filesu
        d = filesv
        
    else:
        files_Central=analyze.get_filenames_15(to,tf, 'central', path)
        files_East=analyze.get_filenames_15(to,tf,'east', path)
        
        i_c = 1
        i_e = 1
        j_c = 1
        j_e = 1
        
        a = files_Central
        b = files_Central
        c = files_East
        d = files_East

    u_u_c, time = analyze.combine_files(a, 'vozocrtx','None',[j_c-1, j_c], [i_c-1,i_c])
    v_v_c, timec = analyze.combine_files(b, 'vomecrty','None',[j_c-1, j_c], [i_c-1,i_c])
    time_c = tidetools.convert_to_seconds(timec)
    dep_t_c = nc.Dataset(b[-1]).variables['depthv']

    u_u_e, time = analyze.combine_files(c, 'vozocrtx','None',[j_e-1, j_e], [i_e-1,i_e])
    v_v_e, timee = analyze.combine_files(d, 'vomecrty','None',[j_e-1, j_e], [i_e-1,i_e])
    time_e = tidetools.convert_to_seconds(timee)
    dep_t_e = nc.Dataset(d[-1]).variables['depthv']

        
    depth=[dep_t_c[:], dep_t_e[:]] 
    
    u_u_0 = np.ma.masked_values(u_u_e, 0)
    v_v_0 = np.ma.masked_values(v_v_e, 0)
    u_u_0c = np.ma.masked_values(u_u_c, 0)
    v_v_0c = np.ma.masked_values(v_v_c, 0)

    u_c, v_c=research_VENUS.unstag_rot_gridded(u_u_0c, v_v_0c, 'Central')
    u_e, v_e=research_VENUS.unstag_rot_gridded(u_u_0, v_v_0, 'East')

    
    
    if depav == 'None':
        us=[u_c, u_e]
        vs=[v_c, v_e]   
        thesize=(40,2)
        
    else: 
        jc = np.where(np.logical_and( depth[0] > depav[0], depth[0] < depav[1]))
        je = np.where(np.logical_and( depth[1] > depav[2], depth[1] < depav[3]))
        
        u_c_slice = u_c[:,jc[0]]
        v_c_slice = v_c[:,jc[0]]
        u_e_slice = u_e[:,je[0]]
        v_e_slice = v_e[:,je[0]]
        
        uc_av = analyze.depth_average(u_c_slice,depth[0][jc],1)
        vc_av = analyze.depth_average(v_c_slice,depth[0][jc],1)
        ue_av = analyze.depth_average(u_e_slice,depth[1][je],1)
        ve_av = analyze.depth_average(v_e_slice,depth[1][je],1)
        
        thesize = (1,2)
        us=[uc_av, ue_av]
        vs=[vc_av, ve_av] 
        
    times=[time_c,time_e]    
    i=np.arange(0,2)

    vM2amp = np.zeros(thesize); vM2pha = np.zeros(thesize)
    vK1amp = np.zeros(thesize); vK1pha = np.zeros(thesize)
    uM2amp = np.zeros(thesize); uM2pha = np.zeros(thesize)
    uK1amp = np.zeros(thesize); uK1pha = np.zeros(thesize) 
    for i, u, time, v in zip(i, us, times, vs):
        uM2amp[:,i], uM2pha[:,i], uK1amp[:,i], uK1pha[:,i] = tidetools.fittit(u, time)
        vM2amp[:,i], vM2pha[:,i], vK1amp[:,i], vK1pha[:,i] = tidetools.fittit(v, time)

    CX, SX, CY, SY, ap, am, ep, em, major, minor, theta, phase = tidetools.ellipse_params (uM2amp, uM2pha, vM2amp, vM2pha)
    CX_k, SX_k, CY_k, SY_k, ap_k, am_k, ep_k, em_k, major_k, minor_k, theta_k, phase_k = tidetools.ellipse_params (uK1amp, uK1pha, vK1amp, vK1pha)

    return depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k

In [4]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2015,6,6)
tf=datetime.datetime(2015,6,8)
freq = 'h'
t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam(to, tf, path, 'h', depav=[35,300,20,160])

In [3]:
def get_filenames_15(t_orig, t_final, station, model_path):
    """Returns a list with the filenames for all files over the
    defined period of time and sorted in chronological order for 
    the gridded 15 minutes data.

    :arg t_orig: The beginning of the date range of interest.
    :type t_orig: datetime object

    :arg t_final: The end of the date range of interest.
    :type t_final: datetime object

    :arg model_path: Defines the path used (eg. nowcast)
    :type model_path: string

    :returns: files, a list of filenames
    """

    numdays = (t_final-t_orig).days
    dates = [t_orig + datetime.timedelta(days=num)
             for num in range(0, numdays+1)]
    dates.sort()
    
    files = []
    for i in dates:
        sdt = i.strftime('%d%b%y').lower()
        filename = (model_path+'{}/VENUS_{}_gridded.nc'.format(sdt, station))
        files.append(filename)
    
    return files

In [4]:
def loadparam_15(to, tf, path):
    """ This function loads all the data between the start and the end date
    that contains gridded quarter-hourly velocity netCDF4 files. Then it mask, unstaggers and 
    rotates the velocities by component about the VENUS nodes. Lastly it fits 
    the velcities and caculates the tidal ellipse parameters for that date range.**(Important not 
    to have a to < 2015-05-09 because the files do not exist before this date).**
    """
    
    files_Central=analyze.get_filenames_15(to,tf, 'central', path)
    files_East=analyze.get_filenames_15(to,tf,'east', path)
    
    u_u_c, timer = analyze.combine_files(files_Central, 'vozocrtx','None',[0,1], [0,1])
    v_v_c, time = analyze.combine_files(files_Central, 'vomecrty','None', [0,1], [0,1])
    time_c = tidetools.convert_to_seconds(timer)
    dep_t_c= nc.Dataset(files_Central[-1]).variables['depthv']

    u_u_e, time = analyze.combine_files(files_East,'vozocrtx', 'None', [0,1], [0,1])
    v_v_e, time = analyze.combine_files(files_East,'vomecrty', 'None',[0,1], [0,1])
    time_e = tidetools.convert_to_seconds(timer)
    dep_t_e = nc.Dataset(files_East[-1]).variables['depthv']
    
    depth=[dep_t_c[:], dep_t_e[:]] 
    
    u_u_0 = np.ma.masked_values(u_u_e, 0)
    v_v_0 = np.ma.masked_values(v_v_e, 0)
    u_u_0c = np.ma.masked_values(u_u_c, 0)
    v_v_0c = np.ma.masked_values(v_v_c, 0)

    u_c, v_c=research_VENUS.unstag_rot_gridded(u_u_0c, v_v_0c, 'Central')
    u_e, v_e=research_VENUS.unstag_rot_gridded(u_u_0, v_v_0, 'East')
    times=[time_c,time_e]
    us=[u_c, u_e]
    vs=[v_c, v_e]
    i=np.arange(0,2)

    thesize=(40,2)
    vM2amp = np.zeros(thesize); vM2pha = np.zeros(thesize)
    vK1amp = np.zeros(thesize); vK1pha = np.zeros(thesize)
    uM2amp = np.zeros(thesize); uM2pha = np.zeros(thesize)
    uK1amp = np.zeros(thesize); uK1pha = np.zeros(thesize)

    for i, u, time, v in zip(i, us, times, vs):
        uM2amp[:,i], uM2pha[:,i], uK1amp[:,i], uK1pha[
            :,i] = tidetools.fittit(u, time)
        vM2amp[:,i], vM2pha[:,i], vK1amp[:,i], vK1pha[
            :,i] = tidetools.fittit(v, time) 

    CX, SX, CY, SY, ap, am, ep, em, major, minor, theta = tidetools.ellipse_params (uM2amp, uM2pha, vM2amp, vM2pha)
   
    CX_k, SX_k, CY_k, SY_k, ap_k, am_k, ep_k, em_k, major_k, minor_k, theta_k = tidetools.ellipse_params (uK1amp, uK1pha, vK1amp, vK1pha)
    return depth, major, minor, theta, major_k, minor_k, theta_k

Function to save tidal parameters in a csv file.

In [8]:
def writetocsv(runname, depth, major, minor,theta, phase, majork1, minork1, thetak1, phasek1, station):
    outfile = runname+'.csv'
    
    if station == 'Central':
        k=0
        with open(outfile, 'wb') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow([
                'Depth', 'Major(M2)', 'Minor(M2)', 'Inc(M2)','Pha(M2)',
                'Major(K1)', 'Minor(K1)', 'Inc(K1)', 'Pha(K1)'
            ])
            for i in np.arange(0,39):
                writer.writerow([depth[k][i], major[i,k], minor[i,k], theta[i,k], phase[i,k], majork1[i,k], minork1[i,k], thetak1[i,k], phasek1[i,k]])

    elif station == 'East':
        k=1
        with open(outfile, 'wb') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow([
                'Depth', 'Major(M2)', 'Minor(M2)', 'Inc(M2)','Pha(M2)',
                'Major(K1)', 'Minor(K1)', 'Inc(K1)', 'Pha(K1)'
            ])
            for i in np.arange(0,39):
                writer.writerow([depth[k][i], major[i,k], minor[i,k], theta[i,k], phase[i,k], majork1[i,k], minork1[i,k], thetak1[i,k], phasek1[i,k]])

    elif major.shape[0] == 40:
        with open(outfile, 'wb') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow([
                'Depth', 'Major(M2)', 'Minor(M2)', 'Inc(M2)','Pha(M2)',
                'Major(K1)', 'Minor(K1)', 'Inc(K1)', 'Pha(K1)'
            ])
            for i in np.arange(0,39):
                writer.writerow([depth[i], major[i], minor[i], theta[i], phase[i], majork1[i], minork1[i], thetak1[i], phasek1[i]])

    else:
        k=0
        with open(outfile, 'wb') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            writer.writerow([
                'Major(M2)', 'Minor(M2)', 'Inc(M2)','Pha(M2)',
                'Major(K1)', 'Minor(K1)', 'Inc(K1)', 'Pha(K1)'
            ])
            mat = (major[0], minor[0], theta[0], phase[0], majork1[0], minork1[0], thetak1[0], phasek1[0])
            writer.writerow(mat)


###All the hourly data

Do not use data before November 26th 2014 for tidal ellipses, the model tides change this day.

In [45]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2014,11,26)
tf=datetime.datetime(2015,6,27)
freq = '15'
t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
depav = [35,300,20,160]
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k = research_VENUS.loadparam_all(to, tf, path, 266, 424, depav=[35,300])
runname1 = '{}_{}_{}_Central_depav'.format(t_o, t_f, freq)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k,'Mone')

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k = research_VENUS.loadparam_all(to, tf, path, 283, 416, depav=[20,160])
runname2 = '{}_{}_{}_East_depav'.format(t_o, t_f, freq)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k,'None')

###Seasonal sensitivity
Winter

In [5]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2014,11,26)
tf=datetime.datetime(2015,2,26)
freq = 'h'
t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k = research_VENUS.loadparam_all(to, tf,path, 266, 424 )
runname1 = '{}_{}_Central_winter'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k = research_VENUS.loadparam_all(to, tf, path, 283, 416)
runname2 = '{}_{}_East_winter'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

Spring

In [7]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2015,2,27)
tf=datetime.datetime(2015,5,27)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k = research_VENUS.loadparam_all(to, tf,path, 266, 424)
runname1 = '{}_{}_Central_spring'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k = research_VENUS.loadparam_all(to, tf, path, 283, 416)
runname2 = '{}_{}_East_spring'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

###For all quarter-hourly values

In [ ]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2015,5,9)
tf=datetime.datetime(2015,6,9)
freq= 'h'

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= loadparam(to, tf, path)
runname1 = '{}_{}_{}_Central'.format(t_o, t_f, freq)
runname2 = '{}_{}_{}_East'.format(t_o, t_f, freq)

In [9]:
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'Central')
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'East')

Then to open and read the columns you could use the lines below:

df = pd.read_csv('20150601_20150608_h_Central.csv')
depth = df.Depth

#Quarter-Hourly Data
##@ VENUS nodes

### The Month of May 2015

The gridded data quarter hourly data only started to be recorded on May 9th.

In [12]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

to=datetime.datetime(2014,11,26)
tf=datetime.datetime(2015,6,26)
freq='h'

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k=loadparam(to, tf, path)
runname1 = '{}_{}_{}_Central'.format(t_o, t_f, freq)
runname2 = '{}_{}_{}_East'.format(t_o, t_f, freq)

writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'Central')
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'East')

#Hourly Data at Other Locations
##Foreman Model Locations
In Foreman et al. (2004) the tidal currents amplitude and phase of their model was compared to observations at 7 locations. For the locations that are contained in the model I will make comparisons with these values. I this notebook I will load the data. See [notebook](http://nbviewer.ipython.org/urls/bitbucket.org/salishsea/analysis/raw/tip/Muriel/TidalEllipsesComparisons.ipynb) for comparison. The locations and numbers are from Foreman et al. (2004)

In [20]:
def loadparam_all(to, tf, path, i, j, depav='None'):
    """ This function loads all the data between the start and the end date
    that contains hourly velocity netCDF4 files. Then it mask, unstaggers and 
    rotates the velocities by component about the VENUS nodes. Lastly it fits 
    the velcities and caculates the tidal ellipse parameters for that date range.
    
    depth : vector [dmin, dmax]
    """

    filesu = analyze.get_filenames(to,tf, '1h', 'grid_U', path)
    filesv=analyze.get_filenames(to,tf,'1h', 'grid_V', path)

    u_u, timer = analyze.combine_files(filesu, 'vozocrtx','None',[j-1, j], [i-1,i])
    v_v, time = analyze.combine_files(filesv, 'vomecrty','None',[j-1, j], [i-1,i])
    time = tidetools.convert_to_seconds(timer)
    print time
    
    dep_t = nc.Dataset(filesu[-1]).variables['depthu'];
    
    u_u_0 = np.ma.masked_values(u_u, 0)
    v_v_0 = np.ma.masked_values(v_v, 0)

    u, v=research_VENUS.unstag_rot(u_u_0, v_v_0, 1 , 1)
   
    if depav == 'None':  
        thesize=(40)
        
    else: 
        j = np.where(np.logical_and(dep_t[:] > depav[0], dep_t[:] < depav[1]))

        u_slice = u[:,j[0]]
        v_slice = v[:,j[0]]
        u = analyze.depth_average(u_slice, dep_t[j], 1)
        v = analyze.depth_average(v_slice, dep_t[j], 1)
        
        thesize = (1)
           
    vM2amp = np.zeros(thesize); vM2pha = np.zeros(thesize)
    vK1amp = np.zeros(thesize); vK1pha = np.zeros(thesize)
    uM2amp = np.zeros(thesize); uM2pha = np.zeros(thesize)
    uK1amp = np.zeros(thesize); uK1pha = np.zeros(thesize) 
    uM2amp[:], uM2pha[:], uK1amp[:], uK1pha[:] = tidetools.fittit(u, time)
    vM2amp[:], vM2pha[:], vK1amp[:], vK1pha[:] = tidetools.fittit(v, time)

    CX, SX, CY, SY, ap, am, ep, em, major, minor, theta, phase = tidetools.ellipse_params (uM2amp, uM2pha, vM2amp, vM2pha)
    CX_k, SX_k, CY_k, SY_k, ap_k, am_k, ep_k, em_k, major_k, minor_k, theta_k, phase_k = tidetools.ellipse_params (uK1amp, uK1pha, vK1amp, vK1pha)

    return dep_t, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k

###35 - Haro Strait

In [8]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'
lon35 = -123.225789
lat35 = 48.537579

grid_B = nc.Dataset('/data/dlatorne/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc')
bathy, X, Y = tidetools.get_bathy_data(grid_B)
j35, i35 = tidetools.find_closest_model_point(lon35, lat35, X, Y, bathy)
print i35, j35

233 312


In [9]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 6, 29)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
depav = [35,300,20,160]

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i35, j35)
runname1 = '{}_{}_Haro'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

In [10]:
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i35, j35, depav=[0, 400])
runname2 = '{}_{}_Haro_depav(0-400)'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

###33 - Juan de Fuca West

In [12]:
lat33 = 48.483
lon33 = -124.713

j33, i33 = tidetools.find_closest_model_point(lon33, lat33, X, Y, bathy)
print i33, j33

8 408


In [13]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 6, 29)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()


depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i33, j33)
runname1 = '{}_{}_JuanW'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

In [14]:
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i33, j33, depav=[0, 400])
runname2 = '{}_{}_JuanW_depav(0-400)'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

###34 - Juan de Fuca East

In [15]:
lat34 = 48.232
lon34 = -123.530

j34, i34 = tidetools.find_closest_model_point(lon34, lat34, X, Y, bathy)
print i34, j34

151 275


In [16]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 6, 29)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i34, j34)
runname1 = '{}_{}_JuanE'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

In [17]:
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i34, j34, depav=[0, 400])
runname2 = '{}_{}_JuanE_depav(0-400)'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

###36 - Seymour Narrows
Seymour Narrows is too narrow. Therefore we encounter problems when unstaggering. The neighbouring cells are masked and this causes problems in the calculation routines. I will choose a slightly different location while trying to maintain similar bathymetry and velocities.

In [18]:
lat36 = 50.135820
lon36 = -125.353403

j36, i36 = tidetools.find_closest_model_point(lon36, lat36, X, Y, bathy)
print i36, j36, bathy[j36, i36]

120 774 54.25


In [19]:
inew36=i36-1
jnew36=j36-2

In [20]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 6, 29)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, inew36, jnew36)
runname1 = '{}_{}_SN'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

In [21]:
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k=research_VENUS.loadparam_all(to, tf, path, inew36, jnew36, depav=[0, 400])
runname2 = '{}_{}_SN_depav(0-400)'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

###38 - Johnstone Strait

In [22]:
lat38 = 50.454991
lon38 = -126.038740

j38, i38 = tidetools.find_closest_model_point(lon38,lat38,X, Y, bathy)
print i38, j38

58 885


In [23]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 6, 29)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i38, j38)
runname1 = '{}_{}_JS'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i38, j38, depav=[0, 400])
runname2 = '{}_{}_JS_depav(0-400)'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')


###37 - Arran Rapids

In [24]:
lat37 = 50.416667
lon37 = -125.133333
j37, i37 = tidetools.find_closest_model_point(lon37,lat37,X, Y, bathy)
print i37, j37

189 814


In [25]:
inew37 = i37+2
jnew37 = j37-1

In [26]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 6, 29)


t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

In [27]:
depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, inew37, jnew37)
runname1 = '{}_{}_AR'.format(t_o, t_f)
writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, inew37, jnew37, depav=[0, 500])
runname2 = '{}_{}_AR_depav(0-400)'.format(t_o, t_f)
writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

##Thalweg Locations

In [16]:
thalweg = np.loadtxt('/data/dlatorne/MEOPAR/tools/bathymetry/thalweg_working.txt', dtype=int, unpack=True)

##Systematic choice of Thalweg Locations

In [17]:
path = '/data/dlatorne/MEOPAR/SalishSea/nowcast/'

In [33]:
for k in [800, 900, 1000, 1100, 1200, 1400, 1500]:
    i = thalweg[1,k]
    j = thalweg[0,k]
    depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i, j)
    runname1 = '{}_{}_thalweg_{}'.format(t_o, t_f, k)
    writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

    depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i, j, depav=[0, 400])
    runname2 = '{}_{}_thalweg_{}_depav(0-400)'.format(t_o, t_f, k)
    writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')


##Haro

In [19]:
to=datetime.datetime(2014, 11, 26)
tf=datetime.datetime(2015, 6, 29)

t_o = to.strftime('%d%b%y').lower()
t_f = tf.strftime('%d%b%y').lower()

for k in np.arange(480,530,4):
    i = thalweg[1,k]
    j = thalweg[0,k]
    depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i, j)
    runname1 = '{}_{}_HaroS_{}'.format(t_o, t_f, k)
    writetocsv(runname1, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')

    depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k= research_VENUS.loadparam_all(to, tf, path, i, j, depav=[0, 400])
    runname2 = '{}_{}_HaroS_{}_depav(0-400)'.format(t_o, t_f, k)
    writetocsv(runname2, depth, major, minor, theta, phase, major_k, minor_k, theta_k, phase_k, 'None')
